### Set api keys

Setup before running this notebook
1. Create a file called `.env` in this directory

2. Write `OPENAI_API_KEY="Your Api Key"` in the .env file

3. Gitignore your `.env` file


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
from openai import OpenAI
import re
import json

In [4]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
def append_txt_files(directory):
    result = ""
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    result += f.read() + "\n"
    return result

In [6]:
SYSTEM_PROMPT = """
You are a helpful assistant used for data processing. You have expert understanding of taking in
chunks of textual data and generating question answer pairs for supervised fine tuning. You do not incorporate your own knowledge
and only use factual information form the provided knowledge text."""

In [4]:
USER_PROMPT = """
I want to create question answer pairs for the following knowledge text, format the pairs like this example:
<example>
<example_knowledge_text>
1.035 Co-Owners (VC §§4150.5 and 9852.5)
A vehicle or vessel may be owned by two or more co-owners. Co-owner names may be joined by “and”, “and/or”, or “or”. All owners mustendorse the title or registration application to register the vehicle/vessel, but the requirements for releasing ownership vary. Refer to Chapter 11.
Certificates issued for applications not indicating “and” or “or” between the names will show “and” as represented by a slash (/) between the names.
* The signatures of all owners are required to transfer ownership when the co-owner names are joined by “and”. Ownership passes to the surviving co-owner upon the death of a co-owner or, with the surviving co-owner’s release, to a new owner. A deceased co-owner’s interest may only be released by one of the following:
   * Heir of the deceased with an Affidavit for Transfer Without Probate California Titled Vehicle or Vessels Only (REG 5) form.
   * Administrator with Letters of Administration.
   * Executor with Letters Testamentary.
* The signature of only one owner is required to transfer ownership when the co-owner names are joined by “and/or” or “or”. A surviving co-owner’s signature on the title releases all owners’ interest unless “Tenants in Common” or “COMPRO” follows the co-owner’s names.
* A REG 5 cannot be used to circumvent the interest of a surviving owner when the vehicles are jointly owned by two or more persons and one of the owners is deceased. However, the surviving owner (if they are the heir) may complete a REG 5 to release the interest of the deceased owner. The California Certificate of Title must be signed twice, once by surviving owner and once for the deceased owner countersigned by the heir. If owned jointly by two or more deceased owners, a REG 5 for the most recently deceased owner and a death certificate for each owner is required.
Tenants in Common—When “Tenants in Common” follows the names of co-owners, the interest of a deceased co-owner reverts to the deceased co-owner’s estate, not to the surviving co-owner. Ownership may be transferred with Letters Testamentary, Letters of Administration, or an Affidavit for Transfer Without Probate California Titled Vehicle or Vessels Only (REG 5) form.
COMPRO—When “COMPRO” (community property) follows the names of co-owners, ownership passes to the surviving owner after the deceased co-owner’s interest is executed by the:
* Heir of the deceased with a REG 5, if the estate was not probated.
* Administrator of the estate with Letters of Administration.
* Executor of the estate with Letters Testamentary.
Ownership may be transferred to a new owner with the surviving co-owner’s release.
JTRS—When “JTRS” (joint tenants with right of survivorship) follows the names of co-owners:
* All owners must release interest during the lifetime of the co-owners.
* Upon the death of an owner, interest is released by the surviving co-owner.
* A copy of the deceased owner’s death certificate must accompany the application.
* The signature of the surviving co-owner(s) on the title releases all owner interests.
TOD (Transfer on Death)—Refer to Chapter 11.
<end_example_knowledge_text>
<example_qa_pair>
[{{'role': 'user', 'content': 'Can I register a car to multiple users in California'}},
{{'role': 'assistant', 'content': 'Absolutely, you can register a car under multiple owners in California. The state allows co-ownership with different options like "and," "or," or "and/or" between names, which affects how ownership transfers work. All co-owners need to sign the initial registration, but transfer requirements vary. There are also special rules for situations like when an owner passes away or for specific ownership types like community property. The key is to choose the right co-ownership option that fits your situation, as it'll determine things like whether you need all owners to sign off on a sale or just one.'}}]
<end_example_qa_pair>
<end_example>

Generate questions that people will commonly ask about this the given knowledge base.
Make sure your answers to the questions are clear and comprehensive, the answers must simulate an expert clearly yet succintly provinding a factual answer.
Make your answers medium lenght, but do not omit important information.

Generate at leat {0} question answer pairs and cover all possible questions that someone might ask about this knowledge text, before genrating each question, say [n/{0}]
Where n is the pair number, make sure to follow the template provided in the example.

so outputs will follow the structure
[0/{0}]
[{{"role": "user", "content:" "..."}}, {{"role": "assistant", "content:" "..."}}]

[1/{0}]
[{{"role": "user", "content:" "..."}}, {{"role": "assistant", "content:" "..."}}]

...
<knowledge_text>
"""

In [5]:
def user_prompt_for(scrape_file):
    knowledge_base = ''
    with open(scrape_file, 'r') as f:
        knowledge_base += f.read() + "\n"
    return USER_PROMPT.format(100) + knowledge_base + "\n<end_knowledge_text>"

In [6]:
print(user_prompt_for("./dataset/scrape_000.txt"))


I want to create question answer pairs for the following knowledge text, format the pairs like this example:
<example>
<example_knowledge_text>
1.035 Co-Owners (VC §§4150.5 and 9852.5)
A vehicle or vessel may be owned by two or more co-owners. Co-owner names may be joined by “and”, “and/or”, or “or”. All owners mustendorse the title or registration application to register the vehicle/vessel, but the requirements for releasing ownership vary. Refer to Chapter 11.
Certificates issued for applications not indicating “and” or “or” between the names will show “and” as represented by a slash (/) between the names.
* The signatures of all owners are required to transfer ownership when the co-owner names are joined by “and”. Ownership passes to the surviving co-owner upon the death of a co-owner or, with the surviving co-owner’s release, to a new owner. A deceased co-owner’s interest may only be released by one of the following:
   * Heir of the deceased with an Affidavit for Transfer Without

In [10]:
def sft_qa_pairs(scrape_file):
    user_prompt = user_prompt_for(scrape_file)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    )
    content = completion.choices[0].message.content
    pattern = r'\[\d+/\d+\]\s+'
    split = re.split(pattern, content)
    split = [item for item in split if item]
    parsed_result = [json.loads(item) for item in split]
    return parsed_result

In [15]:
parsed_result = sft_qa_pairs("./dataset/scrape_007.txt")

In [16]:
print(parsed_result)

[[{'role': 'user', 'content': 'Can multiple people own a vehicle in California?'}, {'role': 'assistant', 'content': "Yes, multiple people can co-own a vehicle in California. Co-owner names can be connected using 'and', 'and/or', or 'or', which defines how ownership rights and responsibilities are shared among the co-owners."}], [{'role': 'user', 'content': 'What happens to vehicle ownership if one co-owner dies?'}, {'role': 'assistant', 'content': "If one co-owner dies, the ownership passes to the surviving co-owner unless specified otherwise. The deceased co-owner's interest can be transferred using specific documents such as an Affidavit for Transfer Without Probate (REG 5) form, Letters of Administration, or Letters Testamentary, depending on the situation."}], [{'role': 'user', 'content': 'How do I transfer ownership of a vehicle co-owned with my sibling if they died?'}, {'role': 'assistant', 'content': "To transfer ownership after the death of a co-owner, you would typically need 

In [17]:
for i in parsed_result: 
    print("QUESTION:", i[0]["content"])
    print("ANSWER:", i[1]["content"])
    print()
    print()

QUESTION: Can multiple people own a vehicle in California?
ANSWER: Yes, multiple people can co-own a vehicle in California. Co-owner names can be connected using 'and', 'and/or', or 'or', which defines how ownership rights and responsibilities are shared among the co-owners.


QUESTION: What happens to vehicle ownership if one co-owner dies?
ANSWER: If one co-owner dies, the ownership passes to the surviving co-owner unless specified otherwise. The deceased co-owner's interest can be transferred using specific documents such as an Affidavit for Transfer Without Probate (REG 5) form, Letters of Administration, or Letters Testamentary, depending on the situation.


QUESTION: How do I transfer ownership of a vehicle co-owned with my sibling if they died?
ANSWER: To transfer ownership after the death of a co-owner, you would typically need to complete a REG 5 form if you're the heir of the deceased. This form allows the surviving owner to release the interest of the deceased in the vehicle